<a href="https://colab.research.google.com/github/carmenhat/carmenhat/blob/main/analisis_tendencias_futbol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from collections import defaultdict, Counter

# Print the current working directory
print("Current working directory:", os.getcwd())

# Cargar los datasets
results_df = pd.read_csv('/results.csv')
goalscorers_df = pd.read_csv('/goalscorers.csv')


# Convertir las fechas a formato datetime
results_df['date'] = pd.to_datetime(results_df['date'])
goalscorers_df['date'] = pd.to_datetime(goalscorers_df['date'])

# Añadir año y período a cada partido
results_df['year'] = results_df['date'].dt.year
results_df['decade'] = (results_df['date'].dt.year // 10) * 10
results_df['period'] = (results_df['date'].dt.year // 5) * 5

goalscorers_df['year'] = goalscorers_df['date'].dt.year
goalscorers_df['period'] = (goalscorers_df['year'] // 5) * 5

# ----- 1. ANÁLISIS POR PORCENTAJE DE VICTORIAS -----

# Función para calcular estadísticas de equipos
def calculate_team_stats(df):
    teams_stats = {}

    # Obtener una lista de todos los equipos únicos
    all_teams = set(df['home_team'].unique()) | set(df['away_team'].unique())

    for team in all_teams:
        # Partidos como local
        home_matches = df[df['home_team'] == team]
        home_wins = len(home_matches[home_matches['home_score'] > home_matches['away_score']])
        home_draws = len(home_matches[home_matches['home_score'] == home_matches['away_score']])
        home_losses = len(home_matches[home_matches['home_score'] < home_matches['away_score']])

        # Partidos como visitante
        away_matches = df[df['away_team'] == team]
        away_wins = len(away_matches[away_matches['away_score'] > away_matches['home_score']])
        away_draws = len(away_matches[away_matches['away_score'] == away_matches['home_score']])
        away_losses = len(away_matches[away_matches['away_score'] < away_matches['home_score']])

        # Total de partidos
        total_matches = len(home_matches) + len(away_matches)
        total_wins = home_wins + away_wins
        total_draws = home_draws + away_draws
        total_losses = home_losses + away_losses

        # Goles
        goals_for = home_matches['home_score'].sum() + away_matches['away_score'].sum()
        goals_against = home_matches['away_score'].sum() + away_matches['home_score'].sum()

        # Calcular porcentajes (evitar división por cero)
        win_percentage = (total_wins / total_matches * 100) if total_matches > 0 else 0

        # Guardar estadísticas
        teams_stats[team] = {
            'matches': total_matches,
            'wins': total_wins,
            'draws': total_draws,
            'losses': total_losses,
            'goals_for': goals_for,
            'goals_against': goals_against,
            'win_percentage': win_percentage
        }

    return teams_stats

# Calcular estadísticas para todos los equipos
all_time_stats = calculate_team_stats(results_df)

# Convertir a DataFrame para mejor manipulación
stats_df = pd.DataFrame.from_dict(all_time_stats, orient='index')

# Filtrar equipos con un mínimo de partidos para evitar anomalías estadísticas
min_matches = 100
filtered_stats_df = stats_df[stats_df['matches'] >= min_matches].copy()

# Ordenar por porcentaje de victorias para encontrar los mejores equipos
top_teams_by_win_percentage = filtered_stats_df.sort_values('win_percentage', ascending=False).head(20).copy()
print("Top 20 equipos por porcentaje de victorias (mínimo 100 partidos):")
print(top_teams_by_win_percentage[['matches', 'wins', 'draws', 'losses', 'win_percentage']])

# ----- 2. ANÁLISIS POR DÉCADA -----

# Calcular estadísticas por década de forma segura
decades_stats = {}

for decade in sorted(results_df['decade'].unique()):
    decade_df = results_df[results_df['decade'] == decade].copy()
    decades_stats[decade] = calculate_team_stats(decade_df)

# Encontrar el mejor equipo de cada década
best_teams_by_decade = {}

for decade, stats in decades_stats.items():
    teams_df = pd.DataFrame.from_dict(stats, orient='index')
    # Filtrar equipos con al menos 20 partidos en la década
    if not teams_df.empty:
        filtered_teams = teams_df[teams_df['matches'] >= 20].copy()
        if not filtered_teams.empty:
            best_team = filtered_teams.sort_values('win_percentage', ascending=False).iloc[0]
            best_teams_by_decade[decade] = {
                'team': best_team.name,
                'matches': best_team['matches'],
                'win_percentage': best_team['win_percentage']
            }

best_teams_decade_df = pd.DataFrame.from_dict(best_teams_by_decade, orient='index')
best_teams_decade_df.index.name = 'decade'
best_teams_decade_df = best_teams_decade_df.reset_index()
print("\nMejor equipo por década (mínimo 20 partidos):")
print(best_teams_decade_df.sort_values('decade'))

# ----- 3. ANÁLISIS DE TORNEOS GANADOS -----

# Identificar los ganadores de torneos importantes
tournament_winners = {}

# Lista de torneos importantes
major_tournaments = [
    'FIFA World Cup',
    'UEFA Euro',
    'Copa America',
    'AFC Asian Cup',
    'CONCACAF Gold Cup',
    'Africa Cup of Nations'
]

for tournament in major_tournaments:
    tournament_df = results_df[results_df['tournament'] == tournament].copy()

    # Agrupar por año para identificar las finales
    tournament_df['year'] = tournament_df['date'].dt.year
    years = tournament_df['year'].unique()

    for year in years:
        year_df = tournament_df[tournament_df['year'] == year].copy()

        # Tomar el último partido como la final
        final_match = year_df.sort_values('date').iloc[-1]

        # Determinar ganador
        if final_match['home_score'] > final_match['away_score']:
            winner = final_match['home_team']
        elif final_match['home_score'] < final_match['away_score']:
            winner = final_match['away_team']
        else:
            # En caso de empate, podríamos buscar información en shootouts_df
            winner = "Empate/No determinado"

        tournament_key = f"{tournament} {year}"
        tournament_winners[tournament_key] = winner

# Contar torneos ganados por equipo
tournament_counts = Counter(tournament_winners.values())
tournament_winners_df = pd.DataFrame.from_dict(tournament_counts, orient='index').reset_index()
tournament_winners_df.columns = ['team', 'tournaments_won']
tournament_winners_df = tournament_winners_df.sort_values('tournaments_won', ascending=False)

print("\nEquipos con más torneos importantes ganados:")
print(tournament_winners_df.head(10))

# ----- 4. SISTEMA ELO SIMPLIFICADO -----

# Implementar un sistema ELO simplificado
def calculate_elo_ratings(df, K=20, initial_rating=1500):
    elo_ratings = defaultdict(lambda: initial_rating)
    elo_history = defaultdict(list)

    # Ordenar partidos por fecha
    sorted_matches = df.sort_values('date')

    for _, match in sorted_matches.iterrows():
        home_team = match['home_team']
        away_team = match['away_team']

        home_rating = elo_ratings[home_team]
        away_rating = elo_ratings[away_team]

        # Calcular probabilidad esperada
        expected_home = 1 / (1 + 10 ** ((away_rating - home_rating) / 400))
        expected_away = 1 - expected_home

        # Determinar resultado real
        if match['home_score'] > match['away_score']:
            actual_home = 1
            actual_away = 0
        elif match['home_score'] < match['away_score']:
            actual_home = 0
            actual_away = 1
        else:
            actual_home = 0.5
            actual_away = 0.5

        # Factor de importancia del partido
        importance = 1
        if 'World Cup' in str(match['tournament']):
            importance = 2  # Mundial tiene más peso
        elif 'Euro' in str(match['tournament']) or 'Copa America' in str(match['tournament']):
            importance = 1.5  # Torneos continentales tienen peso intermedio

        # Actualizar ratings
        elo_ratings[home_team] += K * importance * (actual_home - expected_home)
        elo_ratings[away_team] += K * importance * (actual_away - expected_away)

        # Guardar historial de ratings
        match_date = match['date']
        elo_history[home_team].append((match_date, elo_ratings[home_team]))
        elo_history[away_team].append((match_date, elo_ratings[away_team]))

    return elo_ratings, elo_history

# Calcular ratings ELO
elo_ratings, elo_history = calculate_elo_ratings(results_df)

# Convertir a DataFrame
elo_df = pd.DataFrame.from_dict(elo_ratings, orient='index').reset_index()
elo_df.columns = ['team', 'elo_rating']
elo_df = elo_df.sort_values('elo_rating', ascending=False)

print("\nTop 20 equipos por rating ELO:")
print(elo_df.head(20))

# ----- 5. ANÁLISIS HEAD-TO-HEAD ENTRE MEJORES EQUIPOS -----

# Tomar los 10 mejores equipos según ELO
top_teams = elo_df.head(10)['team'].tolist()

# Crear matriz de enfrentamientos directos
head_to_head_matrix = pd.DataFrame(index=top_teams, columns=top_teams, data=0)

for team1 in top_teams:
    for team2 in top_teams:
        if team1 != team2:
            # Partidos donde team1 es local y team2 visitante
            home_matches = results_df[(results_df['home_team'] == team1) & (results_df['away_team'] == team2)]
            team1_wins_home = len(home_matches[home_matches['home_score'] > home_matches['away_score']])

            # Partidos donde team1 es visitante y team2 local
            away_matches = results_df[(results_df['home_team'] == team2) & (results_df['away_team'] == team1)]
            team1_wins_away = len(away_matches[away_matches['away_score'] > away_matches['home_score']])

            # Total de victorias de team1 sobre team2
            head_to_head_matrix.loc[team1, team2] = team1_wins_home + team1_wins_away

print("\nMatrix de victorias head-to-head entre los 10 mejores equipos:")
print(head_to_head_matrix)

# ----- 6. ANÁLISIS DE VENTAJA LOCAL -----

# Calcular resultados para partidos con local y visitante de forma segura
def calculate_home_advantage(df):
    # Crear un DataFrame para almacenar resultados por período
    periods = sorted(df['period'].unique())
    home_advantage_data = []

    for period in periods:
        period_df = df[df['period'] == period].copy()

        # Partidos en sedes no neutrales
        non_neutral_df = period_df[period_df['neutral'] == False].copy()

        if len(non_neutral_df) > 0:
            # Calcular victorias, empates y derrotas del local
            home_wins = len(non_neutral_df[non_neutral_df['home_score'] > non_neutral_df['away_score']])
            draws = len(non_neutral_df[non_neutral_df['home_score'] == non_neutral_df['away_score']])
            away_wins = len(non_neutral_df[non_neutral_df['home_score'] < non_neutral_df['away_score']])

            total_matches = len(non_neutral_df)
            home_win_pct = (home_wins / total_matches) * 100 if total_matches > 0 else 0
            draw_pct = (draws / total_matches) * 100 if total_matches > 0 else 0
            away_win_pct = (away_wins / total_matches) * 100 if total_matches > 0 else 0

            home_advantage_data.append({
                'period': period,
                'total_matches': total_matches,
                'home_wins': home_wins,
                'draws': draws,
                'away_wins': away_wins,
                'home_win_pct': home_win_pct,
                'draw_pct': draw_pct,
                'away_win_pct': away_win_pct
            })

    return pd.DataFrame(home_advantage_data)

# Análisis de ventaja local por período
home_advantage_df = calculate_home_advantage(results_df)
print("Evolución de la ventaja local por períodos de 5 años:")
print(home_advantage_df[['period', 'total_matches', 'home_win_pct', 'draw_pct', 'away_win_pct']])

# Análisis de partidos en sedes neutrales vs no neutrales
def neutral_vs_non_neutral_analysis(df):
    # Agrupar por período y neutralidad
    neutral_data = []

    for period in sorted(df['period'].unique()):
        period_df = df[df['period'] == period].copy()

        # Partidos en sedes neutrales
        neutral_matches = period_df[period_df['neutral'] == True].copy()

        # Partidos en sedes no neutrales
        non_neutral_matches = period_df[period_df['neutral'] == False].copy()

        # Calcular victorias del "local" en sedes neutrales
        if len(neutral_matches) > 0:
            neutral_home_wins = len(neutral_matches[neutral_matches['home_score'] > neutral_matches['away_score']])
            neutral_home_win_pct = (neutral_home_wins / len(neutral_matches)) * 100
        else:
            neutral_home_win_pct = 0

        # Calcular victorias del local en sedes no neutrales
        if len(non_neutral_matches) > 0:
            non_neutral_home_wins = len(non_neutral_matches[non_neutral_matches['home_score'] > non_neutral_matches['away_score']])
            non_neutral_home_win_pct = (non_neutral_home_wins / len(non_neutral_matches)) * 100
        else:
            non_neutral_home_win_pct = 0

        neutral_data.append({
            'period': period,
            'neutral_matches': len(neutral_matches),
            'non_neutral_matches': len(non_neutral_matches),
            'neutral_home_win_pct': neutral_home_win_pct,
            'non_neutral_home_win_pct': non_neutral_home_win_pct,
            'home_advantage_difference': non_neutral_home_win_pct - neutral_home_win_pct
        })

    return pd.DataFrame(neutral_data)

# Análisis de neutralidad por período
neutral_analysis_df = neutral_vs_non_neutral_analysis(results_df)
print("\nComparación de resultados en sedes neutrales vs no neutrales:")
print(neutral_analysis_df[['period', 'neutral_home_win_pct', 'non_neutral_home_win_pct', 'home_advantage_difference']])

# ----- 7. ANÁLISIS DE GOLES TOTALES -----

def goals_per_match_analysis(df):
    goals_data = []

    for period in sorted(df['period'].unique()):
        period_df = df[df['period'] == period].copy()

        if len(period_df) > 0:
            total_matches = len(period_df)
            total_goals = period_df['home_score'].sum() + period_df['away_score'].sum()
            avg_goals_per_match = total_goals / total_matches

            # Calcular distribución de goles
            home_goals = period_df['home_score'].sum()
            away_goals = period_df['away_score'].sum()
            home_goals_pct = (home_goals / total_goals) * 100 if total_goals > 0 else 0
            away_goals_pct = (away_goals / total_goals) * 100 if total_goals > 0 else 0

            goals_data.append({
                'period': period,
                'total_matches': total_matches,
                'total_goals': total_goals,
                'avg_goals_per_match': avg_goals_per_match,
                'home_goals_pct': home_goals_pct,
                'away_goals_pct': away_goals_pct
            })

    return pd.DataFrame(goals_data)

# Análisis de goles por período
goals_analysis_df = goals_per_match_analysis(results_df)
print("\nEvolución de goles por partido:")
print(goals_analysis_df[['period', 'total_matches', 'total_goals', 'avg_goals_per_match']])

# ----- 8. DISTRIBUCIÓN DE LA FUERZA DE LOS EQUIPOS -----

# Implementar cálculo de ELO por período de forma segura
def calculate_elo_by_period(df, K=20, initial_rating=1500):
    # Períodos únicos en orden cronológico
    periods = sorted(df['period'].unique())

    # Estructura para almacenar ratings ELO al final de cada período
    period_elo_ratings = {}

    # Inicializar ratings
    current_elo = defaultdict(lambda: initial_rating)

    # Procesar partidos en orden cronológico
    sorted_df = df.sort_values('date')

    for period in periods:
        # Filtrar partidos del período actual
        period_df = sorted_df[sorted_df['period'] == period].copy()

        for _, match in period_df.iterrows():
            home_team = match['home_team']
            away_team = match['away_team']

            home_rating = current_elo[home_team]
            away_rating = current_elo[away_team]

            # Calcular probabilidad esperada
            expected_home = 1 / (1 + 10 ** ((away_rating - home_rating) / 400))
            expected_away = 1 - expected_home

            # Determinar resultado real
            if match['home_score'] > match['away_score']:
                actual_home = 1
                actual_away = 0
            elif match['home_score'] < match['away_score']:
                actual_home = 0
                actual_away = 1
            else:
                actual_home = 0.5
                actual_away = 0.5

            # Factor de importancia
            importance = 1
            if 'World Cup' in str(match['tournament']):
                importance = 2
            elif 'Euro' in str(match['tournament']) or 'Copa America' in str(match['tournament']):
                importance = 1.5

            # Actualizar ratings
            current_elo[home_team] += K * importance * (actual_home - expected_home)
            current_elo[away_team] += K * importance * (actual_away - expected_away)

        # Guardar ratings al final del período
        period_elo_ratings[period] = dict(current_elo)

    return period_elo_ratings

# Calcular ELO por período
period_elo_ratings = calculate_elo_by_period(results_df)

# Analizar distribución de ratings ELO por período
def analyze_elo_distribution(period_elo_ratings):
    distribution_data = []

    for period, ratings in period_elo_ratings.items():
        # Convertir a lista
        elo_values = list(ratings.values())

        if elo_values:
            avg_elo = sum(elo_values) / len(elo_values)
            std_elo = np.std(elo_values)
            min_elo = min(elo_values)
            max_elo = max(elo_values)

            # Calcular percentiles para ver la dispersión
            percentiles = np.percentile(elo_values, [25, 50, 75])

            distribution_data.append({
                'period': period,
                'teams_count': len(elo_values),
                'avg_elo': avg_elo,
                'std_elo': std_elo,
                'min_elo': min_elo,
                'max_elo': max_elo,
                'elo_range': max_elo - min_elo,
                'elo_25th': percentiles[0],
                'elo_median': percentiles[1],
                'elo_75th': percentiles[2],
                'iqr': percentiles[2] - percentiles[0]
            })

    return pd.DataFrame(distribution_data)

# Análisis de distribución de ELO
elo_distribution_df = analyze_elo_distribution(period_elo_ratings)
print("\nDistribución de ratings ELO por período:")
print(elo_distribution_df[['period', 'teams_count', 'avg_elo', 'std_elo', 'elo_range', 'iqr']])

# ----- 9. ANÁLISIS DE PARTIDOS DESEQUILIBRADOS -----

def analyze_lopsided_matches(df):
    lopsided_data = []

    # Definir qué consideramos como partido desequilibrado (diferencia de 3+ goles)
    min_goal_difference = 3

    for period in sorted(df['period'].unique()):
        period_df = df[df['period'] == period].copy()

        if len(period_df) > 0:
            # Calcular la diferencia de goles para cada partido de forma segura
            period_df.loc[:, 'goal_difference'] = abs(period_df['home_score'] - period_df['away_score'])

            # Contar partidos desequilibrados
            lopsided_matches = period_df[period_df['goal_difference'] >= min_goal_difference]
            lopsided_count = len(lopsided_matches)
            lopsided_pct = (lopsided_count / len(period_df)) * 100

            # Analizar diferentes niveles de desequilibrio
            high_diff_count = len(period_df[period_df['goal_difference'] >= 5])
            high_diff_pct = (high_diff_count / len(period_df)) * 100

            lopsided_data.append({
                'period': period,
                'total_matches': len(period_df),
                'lopsided_matches': lopsided_count,
                'lopsided_pct': lopsided_pct,
                'high_diff_matches': high_diff_count,
                'high_diff_pct': high_diff_pct
            })

    return pd.DataFrame(lopsided_data)

# Análisis de partidos desequilibrados
lopsided_df = analyze_lopsided_matches(results_df)
print("\nEvolución de partidos desequilibrados (diferencia de 3+ goles):")
print(lopsided_df[['period', 'total_matches', 'lopsided_pct', 'high_diff_pct']])

# ----- 10. ANÁLISIS DE TIPOS DE GOLES -----

def analyze_goal_types(df):
    goal_types_data = []

    for period in sorted(df['period'].unique()):
        period_df = df[df['period'] == period].copy()

        if len(period_df) > 0:
            total_goals = len(period_df)

            # Contar penaltis
            penalty_goals = len(period_df[period_df['penalty'] == True])
            penalty_pct = (penalty_goals / total_goals) * 100 if total_goals > 0 else 0

            # Contar autogoles
            own_goals = len(period_df[period_df['own_goal'] == True])
            own_goal_pct = (own_goals / total_goals) * 100 if total_goals > 0 else 0

            goal_types_data.append({
                'period': period,
                'total_goals': total_goals,
                'penalty_goals': penalty_goals,
                'penalty_pct': penalty_pct,
                'own_goals': own_goals,
                'own_goal_pct': own_goal_pct
            })

    return pd.DataFrame(goal_types_data)

# Análisis de tipos de goles
goal_types_df = analyze_goal_types(goalscorers_df)
print("\nEvolución de tipos de goles (penaltis y autogoles):")
print(goal_types_df[['period', 'total_goals', 'penalty_pct', 'own_goal_pct']])

# ----- EXPORTAR DATOS PARA FLOURISH -----

# 1. Top equipos por porcentaje de victorias
top_teams_by_win_percentage.reset_index().rename(columns={'index': 'team'}).to_csv('top_teams_win_percentage.csv', index=False)

# 2. Mejores equipos por década
best_teams_decade_df.to_csv('best_teams_by_decade.csv', index=False)

# 3. Equipos con más torneos ganados
tournament_winners_df.to_csv('teams_by_tournaments_won.csv', index=False)

# 4. Top equipos por ELO
elo_df.head(20).to_csv('top_teams_by_elo.csv', index=False)

# 5. Matriz de enfrentamientos directos
head_to_head_matrix.reset_index().rename(columns={'index': 'team'}).to_csv('head_to_head_matrix.csv', index=False)

# 6. Ventaja local
home_advantage_df.to_csv('home_advantage_trend.csv', index=False)

# 7. Comparación sedes neutrales vs. no neutrales
neutral_analysis_df.to_csv('neutral_vs_non_neutral.csv', index=False)

# 8. Goles por partido
goals_analysis_df.to_csv('goals_per_match_trend.csv', index=False)

# 9. Distribución de ELO
elo_distribution_df.to_csv('elo_distribution_trend.csv', index=False)

# 10. Partidos desequilibrados
lopsided_df.to_csv('lopsided_matches_trend.csv', index=False)

# 11. Tipos de goles
goal_types_df.to_csv('goal_types_trend.csv', index=False)

# 12. Evolución de ELO para mejores equipos
elo_evolution_data = []
for team in elo_df.head(10)['team'].tolist():
    for date, rating in elo_history[team]:
        elo_evolution_data.append({
            'team': team,
            'date': date,
            'elo_rating': rating
        })

elo_evolution_df = pd.DataFrame(elo_evolution_data)
elo_evolution_df.to_csv('elo_evolution.csv', index=False)

# 13. Datos de goles por año
yearly_goals_df = results_df.groupby('year').agg({
    'home_score': ['sum', 'mean'],
    'away_score': ['sum', 'mean'],
    'home_team': 'count'  # para contar partidos
}).reset_index()

yearly_goals_df.columns = ['year', 'home_goals', 'avg_home_goals', 'away_goals', 'avg_away_goals', 'matches']
yearly_goals_df['total_goals'] = yearly_goals_df['home_goals'] + yearly_goals_df['away_goals']
yearly_goals_df['avg_goals_per_match'] = yearly_goals_df['total_goals'] / yearly_goals_df['matches']

yearly_goals_df.to_csv('yearly_goals_trend.csv', index=False)

Current working directory: /content
Top 20 equipos por porcentaje de victorias (mínimo 100 partidos):
                matches  wins  draws  losses  win_percentage
Jersey              232   152     19      61       65.517241
Brazil             1045   664    214     167       63.540670
Guernsey            240   145     25      70       60.416667
Spain               764   449    176     139       58.769634
Germany            1014   588    212     214       57.988166
Iran                600   343    141     116       57.166667
England            1076   614    257     205       57.063197
Argentina          1053   579    256     218       54.985755
Tahiti              241   131     31      79       54.356846
South Korea         996   531    250     215       53.313253
Italy               873   465    241     167       53.264605
Czech Republic      355   187     72      96       52.676056
Croatia             382   199    103      80       52.094241
Russia              732   376    193     163